In [18]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [19]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Name of the CSV file
file = 'Covid Data.csv'

In [3]:
# The correct encoding must be used to read the CSV in pandas
df = pd.read_csv(file, header=None)

In [4]:
# Preview of the DataFrame
# Note that FIELD8 is likely a meaningless column
df.head()

,0,1,2,3,4,5,6
0,Title: Confirmed COVID-19 Cases and Deaths | KFF,NaN,NaN,NaN,NaN,NaN,NaN
1,Timeframe: as of 9/14/2020 at 2:26PM,NaN,NaN,NaN,NaN,NaN,NaN
2,Location,Number of COVID-19 Cases,"COVID-19 Cases per 1,000,000 Population",Deaths from COVID-19,"COVID-19 Deaths per 1,000,000 Population",COVID-19 Fatality Rate,Footnotes
3,United States,"6,535,105",19910,"194,252",592,0.029724389,1
4,Alabama,"139,459",28443,"2,355",480,0.016886684,NaN


In [5]:
df = df.drop(index=[0, 1])
df.head()

,0,1,2,3,4,5,6
2,Location,Number of COVID-19 Cases,"COVID-19 Cases per 1,000,000 Population",Deaths from COVID-19,"COVID-19 Deaths per 1,000,000 Population",COVID-19 Fatality Rate,Footnotes
3,United States,"6,535,105",19910,"194,252",592,0.029724389,1
4,Alabama,"139,459",28443,"2,355",480,0.016886684,NaN
5,Alaska,"6,343",8671,44,60,0.006936781,NaN
6,Arizona,"208,512",28647,"5,322",731,0.025523711,NaN


In [6]:
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
df.head()

,Location,Number of COVID-19 Cases,"COVID-19 Cases per 1,000,000 Population",Deaths from COVID-19,"COVID-19 Deaths per 1,000,000 Population",COVID-19 Fatality Rate,Footnotes
3,United States,"6,535,105",19910,"194,252",592,0.029724389,1
4,Alabama,"139,459",28443,"2,355",480,0.016886684,NaN
5,Alaska,"6,343",8671,44,60,0.006936781,NaN
6,Arizona,"208,512",28647,"5,322",731,0.025523711,NaN
7,Arkansas,"70,219",23268,981,325,0.013970578,NaN


In [7]:
df.columns

Index(['Location', 'Number of COVID-19 Cases',
       'COVID-19 Cases per 1,000,000 Population', 'Deaths from COVID-19',
       'COVID-19 Deaths per 1,000,000 Population', 'COVID-19 Fatality Rate',
       'Footnotes'],
      dtype='object')

In [8]:
df.columns = ['state', 'number_covid_cases',
       'COVID-19 Cases per 1,000,000 Population', 'number_of_deaths',
       'COVID-19 Deaths per 1,000,000 Population', 'fatality_rate',
       'Footnotes']

In [9]:
# Delete extraneous column
df = df.drop(columns=['COVID-19 Cases per 1,000,000 Population', 'COVID-19 Deaths per 1,000,000 Population', 'Footnotes'], axis=1)
df.head()

,state,number_covid_cases,number_of_deaths,fatality_rate
3,United States,"6,535,105","194,252",0.029724389
4,Alabama,"139,459","2,355",0.016886684
5,Alaska,"6,343",44,0.006936781
6,Arizona,"208,512","5,322",0.025523711
7,Arkansas,"70,219",981,0.013970578


In [10]:
# Identify incomplete rows
df.count()

state                 65
number_covid_cases    57
number_of_deaths      57
fatality_rate         57
dtype: int64

In [11]:
# Drop all rows with missing information
df = df.dropna(how='any')

In [12]:
# Verify dropped rows
df.count()

state                 57
number_covid_cases    57
number_of_deaths      57
fatality_rate         57
dtype: int64

In [13]:
# The Amount column is the wrong data type. It should be numeric.
df.dtypes

state                 object
number_covid_cases    object
number_of_deaths      object
fatality_rate         object
dtype: object

In [14]:
# Use pd.to_numeric() method to convert the datatype of the Amount column
df['number_covid_cases'] = df['number_covid_cases'].str.replace(',', '')
df['number_of_deaths'] = df['number_of_deaths'].str.replace(',', '')
df['number_covid_cases'] = pd.to_numeric(df['number_covid_cases'])
#df['COVID-19 Cases per 1,000,000 Population'] = pd.to_numeric(df['COVID-19 Cases per 1,000,000 Population'])
df['number_of_deaths'] = pd.to_numeric(df['number_of_deaths'])
#df['COVID-19 Deaths per 1,000,000 Population'] = pd.to_numeric(df['COVID-19 Deaths per 1,000,000 Population'])
df.dtypes

state                 object
number_covid_cases     int64
number_of_deaths       int64
fatality_rate         object
dtype: object

In [15]:
df.head()

,state,number_covid_cases,number_of_deaths,fatality_rate
3,United States,6535105,194252,0.029724389
4,Alabama,139459,2355,0.016886684
5,Alaska,6343,44,0.006936781
6,Arizona,208512,5322,0.025523711
7,Arkansas,70219,981,0.013970578


In [16]:
df.to_csv('Covid_clean.csv', index = False)

In [20]:
rds_connection_string = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [21]:
engine.table_names()

['obesity', 'covid_19', 'population']

In [23]:
df.to_sql(name='covid_19', con=engine, if_exists='append', index=False)